In [ ]:
using Plots, DSP, ControlSystems, Interpolations, Interact, Printf

In [ ]:
# Define convolution functionality
function DSP.conv(f, g, t0, t1; N=100, cubic=false)
    dt = (t1 - t0) / N
    t_h, h_ = conv(f, g, t0, t0, dt, t1)
    # Select interpolation algorithm
    interp = cubic ? cubic_spline_interpolation : linear_interpolation
    h = interp(t_h, h_, extrapolation_bc=Line())
end

function DSP.conv(f, g, t0_f, t0_g, dt, t1)
    # Time until which signals have to be computed to get accurate convolution result until t1
    # This assumes that signals vanish before t0_f and t0_g, respectively
    T = t1 - min(t0_f, t0_g) # can be larger than t1 if t0 < 0
    t_f = range(t0_f, step=dt, stop=T)
    N1 = length(t_f)
    t_g = range(t0_g, step=dt, stop=T)
    N2 = length(t_g)
    f_ = f.(t_f)
    g_ = g.(t_g)
    h_ = dt * conv(f_, g_)
    t0_h = t0_f + t0_g
    # interval on which convolution is correct
    # for larger values of t, result can become wrong if input signals have larger support
    N = min(N1, N2)
    t_h = range(t0_h, step=dt, length=N)
    # sanity checks
    if t_h[end] + dt < t1
        @warn "Check convolution interval computation" t_f t_g t_h t1 N1 N2 N
    end
    @assert length(t_h) == N
    t_h, h_[1:N]
end

In [ ]:
# plotting script for convolution result
function plot_convolution(u, g, y, t, plot_t; ylims=:auto)
    u_ = u.(plot_t)
    g_ = g.(plot_t)
    g_flip = g.(t .- plot_t)
    dy = u_ .* g_flip
    p0 = plot(1, label=nothing)
    plot!(plot_t, g_, label="g(τ)", lw=3)
    p1 = plot(plot_t, u_, label="u(τ)", lw=3, ylims=ylims)
    plot!(plot_t, g_flip, label="g(t-τ)", lw=3)
    plot!(plot_t, dy, fill = (0, 0.5, :limegreen), label="u(τ)⋅g(t-τ)")
    p2 = plot(plot_t, y.(plot_t), lw=3, color=:limegreen, label=nothing)
    plot!([t], [y(t)], color=:limegreen, marker=(:c, 6), label="y(t)=u(t)*g(t)", lw=3)
    plot(p0, p1, p2, size=(600, 600), layout=(3, 1))
end

# Define basis signals

In [ ]:
# step function
σ(t) = t >= 0 ? 1.0 : 0.0
# unit rectangle signal
rectangle(t) = σ(t) - σ(t - 1)
# triangular shaped signal
triangle(t) = max(0., 1. - abs(t))

In [ ]:
# Plot basis signals
plot(σ, label="step")
plot!(rectangle, label="rectangle")
plot!(triangle, label="triangle")

Transformation functionality to create more complex signals from basis signals:

In [ ]:
# shift function by t0
shift(t0) = f -> (t -> f(t - t0))
# scale function by s_t and s_y on t and y axis
scale(s_t, s_y) = f -> (t -> s_y * f(t/s_t))
# add two signals
add(f, g) = t -> f(t) + g(t)
# add vector of functions
add(functions) = t -> sum(f(t) for f in functions)

# Exercise 2

In [ ]:
# set plotting range
t0 = -1
t1 = 10
dt = 0.01

plot_t = t0:dt:t1

The same input signal $u$ will be used on multiple systems, which are given by their step response $g$.

In [ ]:
# scale and shift basis rectangle to obtain u
u = rectangle |> scale(3, 1) |> shift(1)
plot(u, plot_t, label="u(t)")

Approximate Dirac impulse as narrow, peaked rectangle.

In [ ]:
ϵ = 0.01
δ = rectangle |> scale(ϵ, 1/ϵ)
δ_shifted = δ |> scale(1, 2.5) |> shift(5)

plot(δ, plot_t, label="Dirac impulse")
plot!(δ_shifted, plot_t, label="Shifted Dirac impulse")

## Exercise 2.1

In [ ]:
# pulse response is scaled and shifted Dirac impulse
g1 = δ |> scale(1, 2.5) |> shift(5)

y1 = conv(u, g1, t0, t1, N=10000)
@manipulate for t = plot_t
    plot_convolution(u, g1, y1, t, plot_t, ylims=(-Inf, 1.5))
end

## Exercise 2.2

In [ ]:
# pulse response is sum of two Dirac impulses
g2 = add(δ |> scale(1, 2.5) |> shift(4), δ |> scale(1, -0.5) |> shift(2))

y2 = conv(u, g2, t0, t1, N=10000)
@manipulate for t = plot_t
    plot_convolution(u, g2, y2, t, plot_t, ylims=(-0.5, 1.5))
end

## Exercise 2.3

In [ ]:
# pulse response is step function (integrator system)
g3 = σ

y3 = conv(u, g3, t0, t1)
@manipulate for t = plot_t
    plot_convolution(u, g3, y3, t, plot_t)
end

## Exercise 2.4

In [ ]:
# pulse response is sum of two rectangles
g4 = add(rectangle |> scale(1, 2) |> shift(1), rectangle |> scale(1, 1) |> shift(2))

y4 = conv(u, g4, t0, t1)
@manipulate for t = plot_t
    plot_convolution(u, g4, y4, t, plot_t)
end

## Exercise 2.5

In [ ]:
# pulse response is PT1 system (low pass filter)
g5(t) = 2 * exp(-t) * σ(t)

y5 = conv(u, g5, t0, t1)
@manipulate for t = plot_t
    plot_convolution(u, g5, y5, t, plot_t, ylims=(-Inf, 2.2))
end